In [ ]:
%pylab inline
%load_ext autoreload
from smartg import smartg, Surface, smartg_read
from profile.profil import Profile
from luts import LUT, Idx, merge, read_lut_hdf, read_mlut_hdf
from smartg_view import semi_polar, smartg_view
from IPython.html.widgets.interaction import interact, interact_manual

In [ ]:
# run SMART-G, and skip if file exists
try:
    output = smartg('SMART-G-PP', THVDEG=30, wl=500., NBPHOTONS=1e9,
                    atm=Profile('afglt'),
                    surf=Surface(),
                   )
except Exception, e:
    output = e.filename

In [ ]:
# read only one dataset
I_up = smartg_read(output, 'I_up (TOA)')

# read all datasets
R = smartg_read(output)
R.print_info()

In [ ]:
%autoreload
smartg_view(R)

In [ ]:
# basic display
semi_polar(I_up)

In [ ]:
# two plots side by side, including transects
fig = figure(figsize=(12, 6))
semi_polar(I_up, index=Idx(5.), rect=221, sub=223, swap=False, fig=fig)
semi_polar(I_up, index=Idx(30.), rect=222, sub=224, swap=True, fig=fig)  # reversed axes

In [ ]:
# interactive display
interact_manual(lambda vmax, phi:
                semi_polar(lut=I_up, index=Idx(phi), vmin=0, vmax=vmax),
                vmax=(0,1.,0.01),
                phi=(0, 180),
               )

-----
# Wavelength loop

In [ ]:
%autoreload
LUTS = []
for wl in linspace(400, 800, 10):
    try: output = smartg('SMART-G-PP', THVDEG=30,
                    wl=wl, NBPHOTONS=1e8,
                    atm=Profile('afglt'),
                    surf=Surface(),
                   )
    except Exception, e: output = e.filename
    print output

    L = smartg_read(output)
    LUTS.append(L)

M = merge(LUTS, ['LAMBDA'])
M['I_up (TOA)'].print_info()
semi_polar(M['I_up (TOA)'].sub[:,0,:], swap=True, index=12)

----
# Difference between two LUTs

In [ ]:
outputs = []
for ex in ['SMART-G-PP', 'SMART-G-SP']:
    try: out = smartg(ex, THVDEG=30, wl=443., NBPHOTONS=1e9,
                         NBTHETA=20., NBPHI=20.,
                        atm=Profile('afglt'))
    except Exception, e: out = e.filename
    outputs.append(out)
    
PP = smartg_read(outputs[0], 'I_up (TOA)')
SP = smartg_read(outputs[1], 'I_up (TOA)')

# show relative difference
semi_polar(100.*(SP-PP)/PP, Idx(45.), fig=figure(figsize(8, 8)), vmin=-2, vmax=2)